In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
from datetime import datetime
import re
import string
import pickle
import requests
import json

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
model = pickle.load(open('./old/modelSVC.pickle','rb'))
tfidf_vectorizer =  pickle.load(open('./old/tfidf_vectorizer.pickle','rb'))
APIurl = 'http://127.0.0.1:3000'


/var/folders/ch/fxrhbppn0ds9grnhwp1j2qfm0000gn/T/ipykernel_47150/1179595794.py:1: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  model = pickle.load(open('./old/modelSVC.pickle','rb'))
/Users/qoisoctava/miniforge3/envs/pySA/lib/python3.9/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator SVC from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/var/folders/ch/fxrhbppn0ds9grnhwp1j2qfm0000gn/T/ipykernel_47150/1179595794.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tfidf_vectorizer =  pickle.load(open('./old/tfidf_vectorizer.pickle','rb'))
/Users/qoisoctava/miniforge3/envs/pySA/lib/python3.9

In [3]:
def scraperKey(keyword,date_since,date_until):
    query = keyword+" lang:id until:"+str(date_until)+" since:"+str(date_since)
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # requests.post('{}/progress/twitter/new'.format(APIurl),{'dateGet':now,'keyword':keyword,'dateSince':date_since,'dateUntil':date_until,'status':1,'source':'tw'} )
    #print(query)
    #print(datetime.now())
    #print("Sedang Mengumpulkan Data Twitter...")
    tweets = []
    while len(tweets) <= 20:
        for tweet in sntwitter.TwitterSearchScraper(query).get_items():
            tweets.append([datetime.now().date(), keyword, tweet.date, tweet.user.username, tweet.content, tweet.hashtags,tweet.mentionedUsers, tweet.likeCount, tweet.retweetCount, tweet.replyCount])
        
        
        
    df = pd.DataFrame(tweets, columns=['timestamp','keyword','date', 'user', 'tweet', 'hashtags', 'mentions', 'likeCount', 'retweetCount', 'replyCount'])
    return df,now


In [4]:
from indoNLP.preprocessing import pipeline, replace_word_elongation, replace_slang, emoji_to_words
pipe = pipeline([emoji_to_words, replace_word_elongation, replace_slang])

In [15]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # remove hashtag
    text = re.sub(r'RT[\s]', '', text) # remove RT
    text = re.sub(r"http\S+", '', text) # remove link
    # text = re.sub(r'[0-9]+', '', text) # remove numbers
    # text = text.encode("ascii", "ignore").decode() #remove emojis

    text = text.replace('\n', ' ') # replace new line into space
    text = [pipe(word) for word in text]
    # text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    # text = text.strip(' ') # remove characters space from both left and right text

    return text

def casefoldingText(text): # Converting all the characters in a text into lower case
    text = text.lower() 
    return text

def tokenizingText(text): # Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text) 
    return text

def filteringText(text): # Remove stopwors in a text
    listStopwords = set(stopwords.words('indonesian'))
    text = [w for w in text if not w in listStopwords]
    return text

def stemmingText(text): # Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    #factory = StemmerFactory()
    from nltk.stem import PorterStemmer
    stemmer = PorterStemmer()
    text = [stemmer.stem(word) for word in text]
    return text

def toSentence(list_words): # Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

alay_dict = pd.read_csv('colloquial-indonesian-lexicon.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 1: 'replacement'})
alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))

def normalize_alay(text):
 text = ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])
 text = re.sub(' +', ' ', text)
 return text



In [6]:
df, now = scraperKey('kereta cepat','2023-01-13','2023-01-18')
topic = 'Test'

/var/folders/ch/fxrhbppn0ds9grnhwp1j2qfm0000gn/T/ipykernel_47150/3958851375.py:11: FutureWarning: content is deprecated, use rawContent instead
  tweets.append([datetime.now().date(), keyword, tweet.date, tweet.user.username, tweet.content, tweet.hashtags,tweet.mentionedUsers, tweet.likeCount, tweet.retweetCount, tweet.replyCount])


In [7]:
now

'2023-01-31 13:22:06'

In [8]:
df

,timestamp,keyword,date,user,tweet,hashtags,mentions,likeCount,retweetCount,replyCount
0,2023-01-31,kereta cepat,2023-01-17 23:16:06+00:00,Baekamee,@sunuy_gans @Adabapaknya1 @worksfess Iyaa kare...,None,"[https://twitter.com/sunuy_gans, https://twitt...",1,0,0
1,2023-01-31,kereta cepat,2023-01-17 22:59:48+00:00,ayuradzan,Early 18 dah dapat lesen kereta dah. kena cepa...,None,None,0,0,1
2,2023-01-31,kereta cepat,2023-01-17 22:20:03+00:00,mase_aga,@Adabapaknya1 @sunuy_gans @worksfess Kalo kere...,None,"[https://twitter.com/Adabapaknya1, https://twi...",1,0,1
3,2023-01-31,kereta cepat,2023-01-17 20:23:14+00:00,Mustafic_Khzan,Penerimaan pajak kita memang besar tapi Yang d...,None,None,0,0,0
4,2023-01-31,kereta cepat,2023-01-17 20:16:39+00:00,dsyauls,@worksfess Harusnya kereta cepat itu Jakarta -...,None,[https://twitter.com/worksfess],0,0,0
...,...,...,...,...,...,...,...,...,...,...
499,2023-01-31,kereta cepat,2023-01-13 01:37:36+00:00,handokosupraja,@endonesiatwit @Dennysiregar7 lu tua geblek pu...,None,"[https://twitter.com/endonesiatwit, https://tw...",0,0,2
500,2023-01-31,kereta cepat,2023-01-13 01:35:11+00:00,mssyahira,"Kereta boleh dapat cepat ni kalau Ativa Av, At...",None,None,2,0,1
501,2023-01-31,kereta cepat,2023-01-13 01:33:53+00:00,mssyahira,Pesanan khidmat masyarakat untuk sesiapa yang ...,None,None,51,28,2
502,2023-01-31,kereta cepat,2023-01-13 00:27:01+00:00,abdul_karel,@Al4mSyahputra Krisis gara2 pinjam duit tdk p...,None,[https://twitter.com/Al4mSyahputra],1,0,0


In [9]:
# id = requests.get('{}/progress/twitter/id/{}'.format(APIurl,now)).json()[0]['id']
print(id)


<built-in function id>


In [10]:
def update(code):
    requests.put('{}/progress/twitter/update'.format(APIurl), data={'id':id,'status': code})

In [16]:

if df.empty:
    # update(404)
    print("Tidak ada data atau query keliru!")
else:
    df = df.drop_duplicates(subset=['tweet'])
    dataset = df
    # update(2)
    #print(dataset)
    print("Sedang Membersihkan Data Twitter...")
    #gabungin hashtags
    # dataset['Hashtags'] = '-'.join(dataset['Hashtags'])
    print(datetime.now())
    print('cleaning text')
    dataset['text_clean'] = dataset['tweet'].apply(cleaningText)
    print(datetime.now())
    print('casefolding text')
    # dataset['text_clean'] = dataset['text_clean'].apply(casefoldingText)
    # dataset['text_clean'] = dataset['text_clean'].apply(normalize_alay)
    
    print(datetime.now())
    print('tokenizing text')
    dataset['text_preprocessed'] = dataset['text_clean'].apply(tokenizingText)
    print(datetime.now())
    print("filtering text")
    # dataset['text_preprocessed'] = dataset['text_preprocessed'].apply(filteringText)
    print(datetime.now())
    print('stemming text')
    # dataset['text_preprocessed'] = dataset['text_preprocessed'].apply(stemmingText)
    # dataset["popularityScore"] = (dataset["likeCount"] + dataset["retweetCount"] + dataset["replyCount"])/3
    print("Selesai Membersihkan Data Twitter :)")
    print(datetime.now())

        ###################################
        ###      SENTIMENT ANALYSIS     ###
        ###################################

    print("Sedang Menganalisis Sentimen Publik...")
    # update(3)

    print(datetime.now())
    # Make text preprocessed (tokenized) to untokenized with toSentence Function
    X = dataset['text_preprocessed'].apply(toSentence)
    X = tfidf_vectorizer.transform(X.values)
    
    # e = X.toarray()
    # #rumus mean
    # n = 0
    # mean = []
    # for i in e:
    #     a = sum(i.tolist())/5000
    #     mean.append(a)

    # #standar deviasi
    # n = 0
    # stdev = []
    # for i in e:
    #     s = np.std(i.tolist())
    #     stdev.append(s)

    # #max value
    # n = 0
    # maks = []
    # for i in e:
    #     a = max(i.tolist())
    #     maks.append(a)

    # #sum
    # n = 0
    # summ = []
    # for i in e:
    #     a = sum(i.tolist())
    #     summ.append(a)

    # #count feature
    # n = 0
    # count = []
    # for i in e:
    #     a = sum(map(lambda x : x != 0, i.tolist()))
    #     count.append(a)

    # extra_X = np.column_stack((e, np.array(mean), np.array(stdev), np.array(maks), np.array(summ), np.array(count)))

    # from scipy import sparse
    # extra_X = sparse.csr_matrix(extra_X)

    y_pred = model.predict(X)
    dataset['sentiment'] = y_pred

    polarity_decode = {0 : 'Negative', 1 : 'Neutral', 2 : 'Positive'}
    dataset['sentiment'] = dataset['sentiment'].map(polarity_decode)
    print("Selesai Menganalisis Sentimen Publik :)")

    print(datetime.now())

    dataset['date'] = pd.to_datetime(dataset['date']).dt.date
    dataset['date']=dataset['date'].astype(str)
        
    print(dataset.dtypes)

    

Sedang Membersihkan Data Twitter...
2023-01-31 13:25:40.462646
cleaning text
2023-01-31 13:25:45.221440
casefolding text
2023-01-31 13:25:45.221537
tokenizing text


TypeError: expected string or bytes-like object

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 481 entries, 0 to 505
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   timestamp          481 non-null    object 
 1   keyword            481 non-null    object 
 2   date               481 non-null    object 
 3   user               481 non-null    object 
 4   tweet              481 non-null    object 
 5   hashtags           51 non-null     object 
 6   mentions           285 non-null    object 
 7   likeCount          481 non-null    int64  
 8   retweetCount       481 non-null    int64  
 9   replyCount         481 non-null    int64  
 10  text_clean         481 non-null    object 
 11  text_preprocessed  481 non-null    object 
 12  popularityScore    481 non-null    float64
 13  sentiment          481 non-null    object 
dtypes: float64(1), int64(3), object(10)
memory usage: 56.4+ KB


In [ ]:
df = dataset[['timestamp', 'keyword', 'date', 'user', 'tweet', 'hashtags', 'mentions', 'likeCount', 'retweetCount', 'replyCount','popularityScore',  'sentiment']]
df

,timestamp,keyword,date,user,tweet,hashtags,mentions,likeCount,retweetCount,replyCount,popularityScore,sentiment
0,2023-01-24,kereta cepat,2023-01-17,Baekamee,@sunuy_gans @Adabapaknya1 @worksfess Iyaa kare...,None,"[https://twitter.com/sunuy_gans, https://twitt...",1,0,0,0.333333,Positive
1,2023-01-24,kereta cepat,2023-01-17,ayuradzan,Early 18 dah dapat lesen kereta dah. kena cepa...,None,None,0,0,1,0.333333,Neutral
2,2023-01-24,kereta cepat,2023-01-17,mase_aga,@Adabapaknya1 @sunuy_gans @worksfess Kalo kere...,None,"[https://twitter.com/Adabapaknya1, https://twi...",1,0,1,0.666667,Neutral
3,2023-01-24,kereta cepat,2023-01-17,Mustafic_Khzan,Penerimaan pajak kita memang besar tapi Yang d...,None,None,0,0,0,0.000000,Negative
4,2023-01-24,kereta cepat,2023-01-17,dsyauls,@worksfess Harusnya kereta cepat itu Jakarta -...,None,[https://twitter.com/worksfess],0,0,0,0.000000,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...
501,2023-01-24,kereta cepat,2023-01-13,handokosupraja,@endonesiatwit @Dennysiregar7 lu tua geblek pu...,None,"[https://twitter.com/endonesiatwit, https://tw...",0,0,2,0.666667,Neutral
502,2023-01-24,kereta cepat,2023-01-13,mssyahira,"Kereta boleh dapat cepat ni kalau Ativa Av, At...",None,None,2,0,1,1.000000,Neutral
503,2023-01-24,kereta cepat,2023-01-13,mssyahira,Pesanan khidmat masyarakat untuk sesiapa yang ...,None,None,51,28,2,27.000000,Neutral
504,2023-01-24,kereta cepat,2023-01-13,abdul_karel,@Al4mSyahputra Krisis gara2 pinjam duit tdk p...,None,[https://twitter.com/Al4mSyahputra],1,0,0,0.333333,Neutral


In [ ]:
df

,timestamp,keyword,date,user,tweet,hashtags,mentions,likeCount,retweetCount,replyCount,popularityScore,sentiment
0,2023-01-24,kereta cepat,2023-01-17,Baekamee,@sunuy_gans @Adabapaknya1 @worksfess Iyaa kare...,None,"[https://twitter.com/sunuy_gans, https://twitt...",1,0,0,0.333333,Positive
1,2023-01-24,kereta cepat,2023-01-17,ayuradzan,Early 18 dah dapat lesen kereta dah. kena cepa...,None,None,0,0,1,0.333333,Neutral
2,2023-01-24,kereta cepat,2023-01-17,mase_aga,@Adabapaknya1 @sunuy_gans @worksfess Kalo kere...,None,"[https://twitter.com/Adabapaknya1, https://twi...",1,0,1,0.666667,Neutral
3,2023-01-24,kereta cepat,2023-01-17,Mustafic_Khzan,Penerimaan pajak kita memang besar tapi Yang d...,None,None,0,0,0,0.000000,Negative
4,2023-01-24,kereta cepat,2023-01-17,dsyauls,@worksfess Harusnya kereta cepat itu Jakarta -...,None,[https://twitter.com/worksfess],0,0,0,0.000000,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...
501,2023-01-24,kereta cepat,2023-01-13,handokosupraja,@endonesiatwit @Dennysiregar7 lu tua geblek pu...,None,"[https://twitter.com/endonesiatwit, https://tw...",0,0,2,0.666667,Neutral
502,2023-01-24,kereta cepat,2023-01-13,mssyahira,"Kereta boleh dapat cepat ni kalau Ativa Av, At...",None,None,2,0,1,1.000000,Neutral
503,2023-01-24,kereta cepat,2023-01-13,mssyahira,Pesanan khidmat masyarakat untuk sesiapa yang ...,None,None,51,28,2,27.000000,Neutral
504,2023-01-24,kereta cepat,2023-01-13,abdul_karel,@Al4mSyahputra Krisis gara2 pinjam duit tdk p...,None,[https://twitter.com/Al4mSyahputra],1,0,0,0.333333,Neutral


In [ ]:
# df[['timestamp','date']] = df[['timestamp','date']].astype(str)

In [ ]:
# api = df.to_json("./test_data/json.json",orient='records')
data_dict = df.to_dict(orient='records')

In [ ]:


for i in data_dict:
    dicts ={
        'dateGet':i['timestamp'],
        'keyword':i['keyword'],
        'contentDate':i['date'],
        'username':i['user'],
        'tweet':i['tweet'],
        'hashtags':i['hashtags'],
        'mentions':i['mentions'],
        'likeCount':i['likeCount'],
        'retweetCount':i['retweetCount'],
        'replyCount':i['replyCount'],
        'popularityScore':i['popularityScore'],
        'sentiment':i['sentiment'],
        'topic':topic
        }
    # print(dicts)
    # c = requests.post('{}/data/twitter'.format(APIurl),dicts )
    print(c)

update(4)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [ ]:
# cleanData[['Tweet', 'Result Prediction', 'text_preprocessed']].to_csv('pemilu.csv')
  


In [ ]:
df_hashtags = df[['keyword','date','hashtags','sentiment']]
df_hashtags = df_hashtags[df_hashtags['hashtags'].notna()].reset_index()
df_mentions = df[['keyword','date','mentions','sentiment']]
df_mentions = df_mentions[df_mentions['mentions'].notna()].reset_index()

In [ ]:
df_mentions = df[['keyword','date','mentions','sentiment']]
df_mentions = df_mentions[df_mentions['mentions'].notna()].reset_index()

In [ ]:
df_mentions

,index,keyword,date,mentions,sentiment
0,0,kereta cepat,2023-01-17,"[https://twitter.com/sunuy_gans, https://twitt...",Positive
1,2,kereta cepat,2023-01-17,"[https://twitter.com/Adabapaknya1, https://twi...",Neutral
2,4,kereta cepat,2023-01-17,[https://twitter.com/worksfess],Neutral
3,8,kereta cepat,2023-01-17,"[https://twitter.com/kuasabu2, https://twitter...",Neutral
4,12,kereta cepat,2023-01-17,[https://twitter.com/worksfess],Neutral
...,...,...,...,...,...
280,497,kereta cepat,2023-01-13,"[https://twitter.com/novykayra, https://twitte...",Neutral
281,498,kereta cepat,2023-01-13,[https://twitter.com/txtdaribogor],Neutral
282,500,kereta cepat,2023-01-13,"[https://twitter.com/Paltiwest, https://twitte...",Neutral
283,501,kereta cepat,2023-01-13,"[https://twitter.com/endonesiatwit, https://tw...",Neutral


In [ ]:
for i in range(df_hashtags.shape[0]):
    for j in range(len(df_hashtags['hashtags'][i])):
        contentDate = df_hashtags['date'][i]
        keyword = df_hashtags['keyword'][i]
        hashtag = df_hashtags['hashtags'][i][j]
        sentiment = df_hashtags['sentiment'][i]
        if hashtag != 0:
            requests.post('{}/data/twitter/content'.format(APIurl),{
                'contentDate' : contentDate,
                'topic' : topic,
                'keyword' : keyword,
                'content' : hashtag,
                'type' : 'hashtag',
                'sentiment' : sentiment
            } )
    
    

In [ ]:
for i in range(df_mentions.shape[0]):
    for j in range(len(df_mentions['mentions'][i])):
        contentDate = df_mentions['date'][i]
        keyword = df_mentions['keyword'][i]
        mention = str(df_mentions['mentions'][i][j]).replace('https://twitter.com/', '@')
        sentiment = df_mentions['sentiment'][i]
        if mention != 0:
            requests.post('{}/data/twitter/content'.format(APIurl),{
                'contentDate' : contentDate,
                'topic' : topic,
                'keyword' : keyword,
                'content' : mention,
                'type' : 'mention',
                'sentiment' : sentiment
            } )
            print(mention)
            
    
    

@sunuy_gans
@Adabapaknya1
@worksfess
@Adabapaknya1
@sunuy_gans
@worksfess
@worksfess
@kuasabu2
@riecho_reeboun
@Dennysiregar7
@worksfess
@kelapakopyorr
@KAI121
@AirinDatangLagi
@worksfess
@HwrNfl
@marwanFC
@worksfess
@CommuterLine
@Yorkeous
@hannajoe20
@karniilyas
@kaesangp
@kaesangp
@ridwankamil
@worksfess
@ridwankamil
@democrazymedia
@sadlybey
@daevus_
@marwanFC
@worksfess
@CommuterLine
@jalur5_
@worksfess
@kemenkomarves
@upikxc
@LuqmanBeeNKRI
@lilcigart
@sunuy_gans
@worksfess
@RuangLucuuu
@HelmiFelis_
@RuangLucuuu
@worksfess
@dwambrrr
@sunuy_gans
@worksfess
@worksfess
@RuangLucuuu
@urcottoncandyy
@worksfess
@tanyakanrl
@BuletinTV3
@worksfess
@andreasstrp
@marwanFC
@worksfess
@CommuterLine
@worksfess
@2TitikHitam
@hannajoe20
@karniilyas
@sunuy_gans
@worksfess
@worksfess
@fauzan69899430
@Dediy_orlando
@Dennysiregar7
@ganjarpranowo
@jokowi
@ridwankamil
@antoanto_
@dododid64453759
@BradHarizz
@worksfess
@KAI121
@KemenBUMN
@fauzan69899430
@Dediy_orlando
@Dennysiregar7
@ganjarpranowo
@jok

In [ ]:

# SELECT  content,
# 	COUNT( IF( sentiment = 'Positive',1,  NULL) ) AS 'positive',
# 	COUNT( IF( sentiment = 'Neutral',1, NULL) ) AS 'neutral',
# 	COUNT( IF( sentiment = 'Negative',1, Null) ) AS 'negative',
# 	COUNT(sentiment) AS 'total'
# FROM twitter_content
# WHERE type = 'mention'
# GROUP BY content
# ORDER BY total desc;